# **Computational Drug Discovery [Part 1] Download Bioactivity Data**

Chanin Nantasenamat

[*'Data Professor' YouTube channel*](http://youtube.com/dataprofessor)

In this Jupyter notebook, we will be building a real-life **data science project** that you can include in your **data science portfolio**. Particularly, we will be building a machine learning model using the ChEMBL bioactivity data.

---

## **ChEMBL Database**

The [*ChEMBL Database*](https://www.ebi.ac.uk/chembl/) is a database that contains curated bioactivity data of more than 2 million compounds. It is compiled from more than 76,000 documents, 1.2 million assays and the data spans 13,000 targets and 1,800 cells and 33,000 indications.
[Data as of March 25, 2020; ChEMBL version 26].

## **Installing libraries**

Install the ChEMBL web service package so that we can retrieve bioactivity data from the ChEMBL Database.

In [1]:
! pip install chembl_webresource_client

## **Importing libraries**

In [4]:
# Import necessary libraries
import pandas as pd
from chembl_webresource_client.new_client import new_client

## **Search for Target protein**

### **Target search for coronavirus**

In [7]:
# Target search for coronavirus
target = new_client.target
target_query = target.search('aromatase')
targets = pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,"[{'xref_id': 'P11511', 'xref_name': None, 'xre...",Homo sapiens,Cytochrome P450 19A1,20.0,False,CHEMBL1978,"[{'accession': 'P11511', 'component_descriptio...",SINGLE PROTEIN,9606
1,"[{'xref_id': 'P22443', 'xref_name': None, 'xre...",Rattus norvegicus,Cytochrome P450 19A1,20.0,False,CHEMBL3859,"[{'accession': 'P22443', 'component_descriptio...",SINGLE PROTEIN,10116


### **Select and retrieve bioactivity data for *SARS coronavirus 3C-like proteinase* (fifth entry)**

We will assign the fifth entry (which corresponds to the target protein, *coronavirus 3C-like proteinase*) to the ***selected_target*** variable 

In [10]:
selected_target = targets.target_chembl_id[4]
selected_target

KeyError: 4

Here, we will retrieve only bioactivity data for *coronavirus 3C-like proteinase* (CHEMBL3927) that are reported as IC$_{50}$ values in nM (nanomolar) unit.

In [ ]:
activity = new_client.activity
res = activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")

In [ ]:
df = pd.DataFrame.from_dict(res)

In [ ]:
df.head(3)

In [ ]:
df.standard_type.unique()

Finally we will save the resulting bioactivity data to a CSV file **bioactivity_data.csv**.

In [ ]:
df.to_csv('bioactivity_data.csv', index=False)

## **Copying files to Google Drive**

Firstly, we need to mount the Google Drive into Colab so that we can have access to our Google adrive from within Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)


Next, we create a **data** folder in our **Colab Notebooks** folder on Google Drive.

In [29]:
! mkdir "/content/gdrive/My Drive/Colab Notebooks/data2"

The syntax of the command is incorrect.


In [31]:
! cp bioactivity_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [33]:
! ls -l "/content/gdrive/My Drive/Colab Notebooks/data"

'ls' is not recognized as an internal or external command,
operable program or batch file.


Let's see the CSV files that we have so far.

In [36]:
! ls

'ls' is not recognized as an internal or external command,
operable program or batch file.


Taking a glimpse of the **bioactivity_data.csv** file that we've just created.

In [39]:
! head bioactivity_data.csv

'head' is not recognized as an internal or external command,
operable program or batch file.


## **Handling missing data**
If any compounds has missing value for the **standard_value** column then drop it

In [42]:
df2 = df[df.standard_value.notna()]
df2

NameError: name 'df' is not defined

Apparently, for this dataset there is no missing data. But we can use the above code cell for bioactivity data of other target protein.

## **Data pre-processing of the bioactivity data**

### **Labeling compounds as either being active, inactive or intermediate**
The bioactivity data is in the IC50 unit. Compounds having values of less than 1000 nM will be considered to be **active** while those greater than 10,000 nM will be considered to be **inactive**. As for those values in between 1,000 and 10,000 nM will be referred to as **intermediate**. 

In [47]:
bioactivity_class = []
for i in df2.standard_value:
  if float(i) >= 10000:
    bioactivity_class.append("inactive")
  elif float(i) <= 1000:
    bioactivity_class.append("active")
  else:
    bioactivity_class.append("intermediate")

NameError: name 'df2' is not defined

### **Iterate the *molecule_chembl_id* to a list**

In [50]:
mol_cid = []
for i in df2.molecule_chembl_id:
  mol_cid.append(i)

NameError: name 'df2' is not defined

### **Iterate *canonical_smiles* to a list**

In [53]:
canonical_smiles = []
for i in df2.canonical_smiles:
  canonical_smiles.append(i)

NameError: name 'df2' is not defined

### **Iterate *standard_value* to a list**

In [56]:
standard_value = []
for i in df2.standard_value:
  standard_value.append(i)

NameError: name 'df2' is not defined

### **Combine the 4 lists into a dataframe**

In [59]:
data_tuples = list(zip(mol_cid, canonical_smiles, bioactivity_class, standard_value))
df3 = pd.DataFrame( data_tuples,  columns=['molecule_chembl_id', 'canonical_smiles', 'bioactivity_class', 'standard_value'])

In [61]:
df3

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value


### **Alternative method**

In [64]:
selection = ['molecule_chembl_id', 'canonical_smiles', 'standard_value']
df3 = df2[selection]
df3

NameError: name 'df2' is not defined

In [66]:
pd.concat([df3,pd.Series(bioactivity_class)], axis=1)

,molecule_chembl_id,canonical_smiles,bioactivity_class,standard_value,0


Saves dataframe to CSV file

In [69]:
df3.to_csv('bioactivity_preprocessed_data.csv', index=False)

In [71]:
! ls -l

'ls' is not recognized as an internal or external command,
operable program or batch file.


Let's copy to the Google Drive

In [74]:
! cp bioactivity_preprocessed_data.csv "/content/gdrive/My Drive/Colab Notebooks/data"

'cp' is not recognized as an internal or external command,
operable program or batch file.


In [76]:
! ls "/content/gdrive/My Drive/Colab Notebooks/data"

'ls' is not recognized as an internal or external command,
operable program or batch file.


---